In [4]:
pip install numpy pandas tensorflow scikit-learn tpot optuna


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import optuna
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

# Import classifiers TPOT might select
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# --------- Load & Preprocess Data ---------
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Use a small subset for faster execution
x_train, y_train = x_train[:3000], y_train[:3000]
x_test, y_test = x_test[:1000], y_test[:1000]

# Flatten & standardize images
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_flat)
x_test_scaled = scaler.transform(x_test_flat)

# Train-validation split
x_train, x_val, y_train, y_val = train_test_split(x_train_scaled, y_train, test_size=0.2, random_state=42)

# --------- Model Selection with TPOT ---------
print("\n Running TPOT for Model Selection...")
tpot = TPOTClassifier(
    generations=1,  # Reduce for speed
    population_size=5,  # Limit number of models
    random_state=42,
    max_time_mins=2,  # Ensure execution finishes within time
    n_jobs=-1
)

x_train_small, y_train_small = x_train[:500], y_train[:500]  # Use only 500 samples
tpot.fit(x_train_small, y_train_small)

# Get the best model
best_pipeline = tpot.fitted_pipeline_
print("\n Best Model Found by TPOT:")
print(best_pipeline)

# Extract classifier from the TPOT pipeline
if isinstance(best_pipeline, Pipeline):
    best_model = best_pipeline.steps[-1][1]
else:
    best_model = best_pipeline

best_model_type = type(best_model)
print(f"\n TPOT Selected Model: {best_model_type.__name__}")

# --------- Hyperparameter Optimization with Optuna ---------
def objective(trial):
    model_params = {}

    # Fine-tune the selected model dynamically
    if isinstance(best_model, RandomForestClassifier):
        model_params["n_estimators"] = trial.suggest_int("n_estimators", 50, 150)
        model_params["max_depth"] = trial.suggest_int("max_depth", 3, 20)
    
    elif isinstance(best_model, SVC):
        model_params["C"] = trial.suggest_float("C", 0.1, 10.0, log=True)
        model_params["kernel"] = trial.suggest_categorical("kernel", ["linear", "rbf"])
    
    elif isinstance(best_model, GaussianNB):
        model_params["var_smoothing"] = trial.suggest_float("var_smoothing", 1e-9, 1e-6, log=True)

    elif isinstance(best_model, KNeighborsClassifier):
        model_params["n_neighbors"] = trial.suggest_int("n_neighbors", 3, 15)
        model_params["weights"] = trial.suggest_categorical("weights", ["uniform", "distance"])

    elif isinstance(best_model, GradientBoostingClassifier):
        model_params["n_estimators"] = trial.suggest_int("n_estimators", 50, 150)
        model_params["learning_rate"] = trial.suggest_float("learning_rate", 0.01, 0.5, log=True)
        model_params["max_depth"] = trial.suggest_int("max_depth", 3, 10)

    else:
        # Generic tuning for unknown models
        params_to_tune = best_model.get_params().keys()
        if "n_estimators" in params_to_tune:
            model_params["n_estimators"] = trial.suggest_int("n_estimators", 50, 150)
        if "max_depth" in params_to_tune:
            model_params["max_depth"] = trial.suggest_int("max_depth", 3, 20)
        if "learning_rate" in params_to_tune:
            model_params["learning_rate"] = trial.suggest_float("learning_rate", 0.01, 0.5, log=True)

    model = best_model.__class__(**model_params)
    model.fit(x_train[:2000], y_train[:2000])  # Use 2000 samples for speed
    y_pred = model.predict(x_val)
    return accuracy_score(y_val, y_pred)

print("\n Running Optuna for Hyperparameter Tuning...")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)  # Reduce trials for speed

print("\n Best Hyperparameters from Optuna:", study.best_params)
best_params = study.best_params

# Train final optimized model
final_model = best_model.__class__(**best_params)
final_model.fit(x_train, y_train)

# Evaluate final model
y_test_pred = final_model.predict(x_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"\n Test Accuracy with Optimized Hyperparameters: {test_accuracy:.4f}")

# --------- Plot & Save Optimization History ---------
trials_df = study.trials_dataframe()

plt.figure(figsize=(8, 5))
plt.plot(trials_df['number'], trials_df['value'], label='Validation Accuracy', marker='o')
plt.title("Optuna Hyperparameter Optimization History")
plt.xlabel("Trial Number")
plt.ylabel("Validation Accuracy")
plt.legend()
plt.grid(True)

# Save plot
plot_filename = "optuna_optimization_plot.png"
plt.savefig(plot_filename, bbox_inches="tight")
plt.close()

# Save HTML report
html_filename = "optuna_optimization_plot.html"
with open(html_filename, "w", encoding="utf-8") as f:
    f.write(f"<html><body><h1>Optuna Hyperparameter Optimization Plot</h1><img src='{plot_filename}'></body></html>")

print(f"\n Optimization plot saved as {html_filename}.")



 Running TPOT for Model Selection...
is_classifier


f:\Aarzoo\Mtech AI\Recordings\sem3\MLOPS\Assignment 2\MLOPS-bits-Group41-Assignment2\venv\Lib\site-packages\sklearn\base.py:1230: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(
f:\Aarzoo\Mtech AI\Recordings\sem3\MLOPS\Assignment 2\MLOPS-bits-Group41-Assignment2\venv\Lib\site-packages\sklearn\base.py:1270: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(


is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_classifier
is_classifier
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_classifier
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier


Version 0.12.2 of tpot is outdated. Version 1.0.0 was released Wednesday February 26, 2025.
[I 2025-03-23 17:39:51,126] A new study created in memory with name: no-name-870ed6d4-48af-4b19-8b75-10c2365ea287



 Best Model Found by TPOT:
Pipeline(steps=[('extratreesclassifier',
                 ExtraTreesClassifier(criterion='entropy',
                                      max_features=0.6000000000000001,
                                      min_samples_leaf=7, min_samples_split=3,
                                      random_state=42))])

 TPOT Selected Model: ExtraTreesClassifier

 Running Optuna for Hyperparameter Tuning...


[I 2025-03-23 17:39:54,173] Trial 0 finished with value: 0.8316666666666667 and parameters: {'n_estimators': 136, 'max_depth': 16}. Best is trial 0 with value: 0.8316666666666667.
[I 2025-03-23 17:39:54,955] Trial 1 finished with value: 0.84 and parameters: {'n_estimators': 77, 'max_depth': 15}. Best is trial 1 with value: 0.84.
[I 2025-03-23 17:39:57,752] Trial 2 finished with value: 0.8266666666666667 and parameters: {'n_estimators': 138, 'max_depth': 14}. Best is trial 1 with value: 0.84.
[I 2025-03-23 17:39:58,546] Trial 3 finished with value: 0.7883333333333333 and parameters: {'n_estimators': 56, 'max_depth': 8}. Best is trial 1 with value: 0.84.
[I 2025-03-23 17:39:59,119] Trial 4 finished with value: 0.8066666666666666 and parameters: {'n_estimators': 52, 'max_depth': 10}. Best is trial 1 with value: 0.84.



 Best Hyperparameters from Optuna: {'n_estimators': 77, 'max_depth': 15}

 Test Accuracy with Optimized Hyperparameters: 0.8320

 Optimization plot saved as optuna_optimization_plot.html.


Observations -
* TPOT is an AutoML Library that explores different models and pipelines to find the best-performing one based on a specified metric(accuracy in this case). It is comparatively more straightforeard and based on genetic algortithm approach which helps to explore a larger and more diverse search space, potentially finding better model-pipeline contributions.
* From the above logs, TPOT has identified ExtraTreesClassifier as the best model for the fashion MNIST dataset. The classifier was found with specific hyperparameters such as criterion='entropy',  max_features=0.6,min_samples_leaf=7, min_samples_split=3 and a defined random_state. These settings maximize accuracy based on TPOT's search over multiple models and pipelines.
* Optuna was further used to tune the resultant hyperparameters of the chosen ExtraTreesClassifier. The logs shows that multiple trials were conducted to optimize the hyperparameters like n_estimators(number of trees in the forest) and max_depth(maximum depth of the trees).
* The best set of hyperparameters found were n_estimators=77 and max_depth=15 yielding a test accuracy of 83.2%.
* Compared to HyperOpt and RayTune, Optuna genrally gives more flexible and offers better visualization capabilities(like optuna_optimization_plot.html). It is also known for its faster convergence in many scenarios. It is also comparatively simpler-to-use API and more efficient trial pruning making it a better choice.